In [1]:
import geopandas
import pandas as pd
import glob
import shapefile
from tqdm import tqdm
import numpy as np
import pickle
import xlrd3
import os
import glob
import openpyxl

In [2]:
# read poi and locations from shp files
green_poi_file = '/home/yangwenhao/local/project/weibo_filter/app/data/green_v0.3.xls'
full_names = pd.read_excel(green_poi_file, sheet_name=0)

In [13]:
poi2fid = {}
fid2name_area = {}
for x in full_names[['FID', 'Name', 'areaaa']].iterrows():
    fid = x[1].FID
    name = x[1].Name.split('/')
    fid2name_area[fid] = [x[1].Name, x[1].areaaa]
    
    for n in name:
        poi2fid[n] = fid

In [19]:
poi2fid_key = set(list(poi2fid.keys()))

In [44]:
# create place2index files giving tweets and locations
# pickle file: [places, sparse_matirx]

# sub_names = ['商业娱乐', '文化展览', '科研教育', '体育健身', '住区', '旅游景点', '企业办公', '公共服务', '交通设施', '医养设施']
for sht_idx in range(0, 1):
    # full_names = pd.read_excel(poi_file, sheet_name=sht_idx)
    # names = full_names.名称
    location_names = set(list(poi2fid.keys()))
    numofprint = 4

    for lst in  ['list2_woimg']: #['list2_img']:
        text_csv = 'all_list/' + '%s.csv'%(lst)
        if not os.path.exists(text_csv):
            data = xlrd3.open_workbook('/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xlsx')
            table = data.sheet_by_index(0)#.sheets()[0]          #通过索引顺序获取
            # get full-text tweets
            texts = []
            if lst == 'first_2w':
                test_idx = 5
            elif lst == 'list2_img':
                test_idx = 4
            elif lst == 'list2_woimg':
                test_idx = 6
            else:
                test_idx = 6

            for i in range(1, table.nrows):
                texts.append(table.row(i)[test_idx].value)

            texts = pd.DataFrame(texts, columns=['text'])
            texts.to_csv('%s.csv'%(lst), index=False)
        else:
            texts = pd.read_csv(text_csv, index_col=0)
            
        # match all palces for all lines
        print("Num of Tweet: ", len(texts))
        places,sparse_matirx = [],[]
        for idx, i in tqdm(enumerate(location_names), ncols=50,
                           total=len(location_names)): #[297604:]
            try:
                texts.text
                result = texts.text.str.contains(i, na=False) # 12:26:26 na to replace Nan
                if result.to_numpy().sum() > 0:
                    places.append(i)
                    sparse_matirx.append(result.to_numpy())
                # else:
                #     print(i, end=' ')
            except Exception as e:
                print(result.to_numpy().sum())
                break

        # if (idx+1) == len(location_names):
        print("Num of POI: ", len(places))
        with open('all_list3_green/pla2idx_%s.pickle' % (lst), 'wb') as f:
            pickle.dump([places, sparse_matirx], f)
        places,sparse_matirx = [],[]

Num of Tweet:  176968


100%|███████████| 237/237 [00:10<00:00, 23.69it/s]

Num of POI:  63


In [57]:
# texts = pd.read_csv('all_list/list2_woimg.csv', index_col=0)
total = 0
for i in poi2fid_key:
    this_num = texts.text.fillna("").str.contains(i).to_numpy().sum()
    total += this_num
    if this_num>0:
        print(i, this_num)
    
print(total)

滨海公园 2
绿岛公园 1
朝阳公园 3
中心广场 3
官港森林公园 2
航母 83
南湖游乐园 1
天津文化中心 14
北宁公园 17
世纪钟广场 3
中心公园 6
桃花园 3
宁园 13
七里海国家湿地公园 3
睦南公园 8
黄崖关长城 20
文化公园 5
南翠屏公园 5
乐园 103
怡园 3
人民公园 23
河西公园 2
中山公园 7
二宫公园 2
桥园公园 1
中山门公园 1
文化广场 9
儿童公园 2
秋实园 1
国际温泉高尔夫俱乐部 3
解放北园 1
李叔同故居纪念馆 3
紫御园 3
海河外滩公园 3
南开公园 2
人民广场 2
蓟县国家地质公园 4
热带植物观光园 1
河东公园 2
金钢公园 1
欢乐谷 105
长泰园 2
水上公园 71
翠湖公园 1
月牙河公园 4
滨海航母主题公园 3
东疆建设开发纪念公园 1
王串场公园 2
玉龙滑雪场 1
解放南园 3
明珠园 2
湿地公园 9
泰丰公园 2
杨柳青广场 1
塘沽森林公园 5
河滨公园 3
燕园 5
动物园 130
凯旋王国 2
紫云公园 1
西沽公园 10
月季园 2
幸福公园 5
741


In [58]:
texts.text.fillna("").str.contains("古文化街").to_numpy().sum()

173

In [51]:
# save match results with pickle
headers = ["相关地点数量", "微博id", "微博主页", "文本", "日期", "图片", "转发数", "评论数", "点赞数"]
# names2loc_key = set(list(names2loc.keys()))
sub_names = ['Business_Entertainment', 'Cultural_Exhibition', 'Education', 'Sports', 'Residential_District', 'Tourist_Attraction', 'Financial_Office', 'Public_Service', 'Transport', 'Medical_Facilities']

# for lst in ['list2_img', 'list2_woimg']:
for lst in ['list2_woimg']:
    text_csv = 'all_list/' + '%s.csv'%(lst)
    text_pickle = 'all_list3_green/' + 'pla2idx_%s.pickle'%(lst)
    text_pickles = glob.glob(text_pickle)
    this_texts = pd.read_csv(text_csv, index_col=0)
    
    for t in text_pickles:
        append_cells = {}
        with open(t, 'rb') as f:
            [places, sparse_matirx] = pickle.load(f)
        
        places_np = np.array(places)
        sparse_np = np.array(sparse_matirx) #.shape
        
        if places_np.shape[0] > 0:
            for j, l in zip(this_texts.index, range(sparse_np.shape[1])):
                row_cells = set([])
                for i in places_np[np.where(sparse_np[:, l] == True)[0]]:
                    if i in poi2fid_key:
                        fid = poi2fid[i]
                        row_cells.add(fid)
                        
                append_cells[j] = row_cells
        else:
            print('Skip %s ...'%(t))
            continue
            
        if lst == 'list2_img':
            read_cols = [2, 3, 4, 5, 6, 7, 8,  9]
        elif lst == 'list2_woimg':
            read_cols = [2, 3, 6, 7, 8, 9, 10, 11]

        read_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xlsx'
        # title_name = sub_names[int(t.split('/')[1].split('_')[0])-1]
        # print(title_name, "...")
        save_lst = 'all_list3_green/' + lst + '_poi.xlsx' # t.replace('pickle', 'loc) 
        read_writebook = xlrd3.open_workbook(filename=read_lst)
        writebook = openpyxl.Workbook()
        # 添加sheet 写入excel, 参数对应 行, 列, 值
        sheet      = writebook.create_sheet(index=0)
        sht_row = 2 # openpyxl start from 1 instead of 0
        # 相关地点数量	微博id	微博主页	文本	日期	图片	转发数	评论数	点赞数
        for i, h in enumerate(headers):
            sheet.cell(1, i+1, h)

        # write header
        # find the tweet with the largest number of locations
        max_los = 1
        for i in append_cells:
            max_los = max(max_los, len(append_cells[i]))
            
        sheet.cell(1, len(headers)+1, '地点')
        sheet.cell(1, len(headers)+2, 'FID')
        sheet.cell(1, len(headers)+3, '面积')

        read_table = read_writebook.sheet_by_index(0)
        nrows = read_table.nrows
        assert (nrows-1) == len(append_cells), print("rows in excel: ", (nrows-1), "rows in loc_csv: ", len(append_cells))
        
        for i in tqdm(range(1, nrows), ncols=50):
            this_line = read_table.row_slice(i)
            fids = append_cells[i-1]
            # locs_names = [l[0] for l in this_locs]

            if len(fids) > 0:
                this_row = sht_row

                for fid in fids:
                    sheet.cell(this_row, 1, len(fids))
                    
                    for j, c in enumerate(read_cols):
                        sheet.cell(this_row, j+2, this_line[c].value)
                    
                    name, area = fid2name_area[fid]   
                    sheet.cell(this_row, len(headers)+1,   name)
                    sheet.cell(this_row, len(headers)+2,   fid)
                    sheet.cell(this_row, len(headers)+3,   area)
                    
                    this_row += 1
                    sht_row = this_row

        writebook.save(save_lst)
        print(save_lst, " completed with {} lines!".format(sht_row))

100%|█| 176968/176968 [00:00<00:00, 185930.03it/s]


all_list3_green/list2_woimg_poi.xlsx  completed with 741 lines!
